### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [1]:
import pandas as pd
import requests
import bs4
from shapely.geometry import Point
import geopandas as gpd
import argparse
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
# una vez cargados los valores, podemos usarlos
EMAIL = os.getenv("EMAIL")
emt_pass = os.getenv("emt_pass")

In [3]:
#probando a conectarme a la api
#obteniendo el token
url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'
headers = {
    'email': EMAIL,
    'password': emt_pass
}

response2 = requests.get(url, headers=headers)
response2_json = response2.json()
accessToken = response2_json['data'][0]['accessToken']
print(accessToken)
#accessToken = 'b7170b89-72aa-4d20-b7d5-d8ce46e5de9e'

b7170b89-72aa-4d20-b7d5-d8ce46e5de9e


In [4]:
#funcion para traer el data frame de bicimad a tiempo real
def get_data():
    token = accessToken
    url = "https://openapi.emtmadrid.es/v1/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [5]:
response3_json = get_data()
response3_json.keys()
pull_response3_json = response3_json['data']
df_response3 = pd.json_normalize(pull_response3_json)
df_response3.head()

,activate,address,dock_bikes,free_bases,id,light,name,no_available,number,reservations_count,total_bases,virtualDelete,geofenced_capacity,geometry.type,geometry.coordinates
0,1,"Avenida del Ensanche de Vallecas, 9,",22,5,2190,1,"453 - Avenida del Ensanche de Vallecas, 9",0,453,0,27,False,20.0,Point,"[-3.61253028, 40.37043968]"
1,1,"Paseo de la Chopera,33,Comunidad de Madrid España",19,4,2205,1,"267 - Paseo de la Chopera, 33",0,267,0,23,False,20.0,Point,"[-3.7003996647238746, 40.39500000000003]"
2,1,"Paseo de la Castellana nº 122,",2,21,2224,0,150 - Castellana frente a Hermanos Pinzón,0,150,0,23,False,20.0,Point,"[-3.6908, 40.4491]"
3,1,"Paseo de la Castellana nº 164,Comunidad de Mad...",7,16,2225,0,157- Castellana 164,0,157,0,23,False,20.0,Point,"[-3.6894151, 40.4591366]"
4,1,"Guetaria , 84b,",18,6,2226,1,395b - 395 - Guetaria 84b,0,395b,0,24,False,20.0,Point,"[-3.71569102, 40.36916772]"


In [6]:
type(df_response3)

pandas.core.frame.DataFrame

In [7]:
df_response3.columns

Index(['activate', 'address', 'dock_bikes', 'free_bases', 'id', 'light',
       'name', 'no_available', 'number', 'reservations_count', 'total_bases',
       'virtualDelete', 'geofenced_capacity', 'geometry.type',
       'geometry.coordinates'],
      dtype='object')

In [8]:
type(df_response3['geometry.coordinates'][0])

list

In [9]:
df_response3['geometry.coordinates'][0]

[-3.61253028, 40.37043968]

In [10]:
df_response3['latitude'] = df_response3.apply(lambda row: row['geometry.coordinates'][1], axis = 1)
df_response3['latitude']

0      40.370440
1      40.395000
2      40.449100
3      40.459137
4      40.369168
         ...    
583    40.415606
584    40.385412
585    40.443133
586    40.441300
587    40.475636
Name: latitude, Length: 588, dtype: float64

In [11]:
df_response3['longitude'] = df_response3.apply(lambda row: row['geometry.coordinates'][0], axis = 1)
df_response3['longitude']

0     -3.612530
1     -3.700400
2     -3.690800
3     -3.689415
4     -3.715691
         ...   
583   -3.709508
584   -3.688461
585   -3.617832
586   -3.708300
587   -3.583887
Name: longitude, Length: 588, dtype: float64

In [12]:
#creamos una función que nos cree el enlace a google maps
def google_maps(lista):
    return "https://www.google.com/maps?q="+str(lista[1])+","+str(lista[0])
df_response3['Google Maps'] = df_response3.apply(lambda row: google_maps(row['geometry.coordinates']),axis = 1)

In [13]:
#google_maps("40.37044","-3.612530")

In [14]:
df_response3.head()

,activate,address,dock_bikes,free_bases,id,light,name,no_available,number,reservations_count,total_bases,virtualDelete,geofenced_capacity,geometry.type,geometry.coordinates,latitude,longitude,Google Maps
0,1,"Avenida del Ensanche de Vallecas, 9,",22,5,2190,1,"453 - Avenida del Ensanche de Vallecas, 9",0,453,0,27,False,20.0,Point,"[-3.61253028, 40.37043968]",40.370440,-3.612530,"https://www.google.com/maps?q=40.37043968,-3.6..."
1,1,"Paseo de la Chopera,33,Comunidad de Madrid España",19,4,2205,1,"267 - Paseo de la Chopera, 33",0,267,0,23,False,20.0,Point,"[-3.7003996647238746, 40.39500000000003]",40.395000,-3.700400,https://www.google.com/maps?q=40.3950000000000...
2,1,"Paseo de la Castellana nº 122,",2,21,2224,0,150 - Castellana frente a Hermanos Pinzón,0,150,0,23,False,20.0,Point,"[-3.6908, 40.4491]",40.449100,-3.690800,"https://www.google.com/maps?q=40.4491,-3.6908"
3,1,"Paseo de la Castellana nº 164,Comunidad de Mad...",7,16,2225,0,157- Castellana 164,0,157,0,23,False,20.0,Point,"[-3.6894151, 40.4591366]",40.459137,-3.689415,"https://www.google.com/maps?q=40.4591366,-3.68..."
4,1,"Guetaria , 84b,",18,6,2226,1,395b - 395 - Guetaria 84b,0,395b,0,24,False,20.0,Point,"[-3.71569102, 40.36916772]",40.369168,-3.715691,"https://www.google.com/maps?q=40.36916772,-3.7..."


In [15]:
for valor in df_response3['Google Maps']:
    print(valor)

https://www.google.com/maps?q=40.37043968,-3.61253028
https://www.google.com/maps?q=40.39500000000003,-3.7003996647238746
https://www.google.com/maps?q=40.4491,-3.6908
https://www.google.com/maps?q=40.4591366,-3.6894151
https://www.google.com/maps?q=40.36916772,-3.71569102
https://www.google.com/maps?q=40.4956969,-3.7249527
https://www.google.com/maps?q=40.39229225,-3.7405438
https://www.google.com/maps?q=40.38091762,-3.72783615
https://www.google.com/maps?q=40.4648762,-3.63135715
https://www.google.com/maps?q=40.39497294,-3.76815631
https://www.google.com/maps?q=40.39659757,-3.77057541
https://www.google.com/maps?q=40.43699597,-3.61191521
https://www.google.com/maps?q=40.37562288,-3.7509733
https://www.google.com/maps?q=40.37313777,-3.74765739
https://www.google.com/maps?q=40.478746,-3.670588
https://www.google.com/maps?q=40.48027409,-3.71869067
https://www.google.com/maps?q=40.48194,-3.703918
https://www.google.com/maps?q=40.48178815,-3.7132206
https://www.google.com/maps?q=40.484327

In [16]:
#Nos traemos el data frame de bicipark
#accessToken = 'b7170b89-72aa-4d20-b7d5-d8ce46e5de9e'
def get_data_2():
    token = accessToken
    url2 = "https://openapi.emtmadrid.es/v1/transport/bicipark/stations/"
    headers2 = {"accessToken" : token}
    response4 = requests.get(url2, headers = headers2)
    return response4.json()

In [17]:
response4_json = get_data_2()
response4_json.keys()
pull_response4_json = response4_json['data']
df_response4 = pd.json_normalize(pull_response4_json)
df_response4.head()

,stationId,integratorId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates
0,74,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,10,1,0,Point,"[-3.6630679, 40.4846838]"
1,22,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,8,1,1,Point,"[-3.693897, 40.452973]"
2,49,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,10,1,0,Point,"[-3.690786, 40.418146]"
3,12,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,9,1,0,Point,"[-3.692879, 40.430361]"
4,5,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,28036,13,11,1,0,Point,"[-3.678921, 40.472419]"


In [18]:
df_response4['latitude'] = df_response4.apply(lambda row: row['geometry.coordinates'][1], axis = 1)
df_response4['latitude']

0     40.484684
1     40.452973
2     40.418146
3     40.430361
4     40.472419
5     40.430315
6     40.425608
7     40.495942
8     40.383492
9     40.417220
10    40.400306
11    40.415820
12    40.434302
13    40.420779
14    40.411138
15    40.414464
16    40.431005
17    40.422856
18    40.432409
Name: latitude, dtype: float64

In [19]:
df_response4['longitude'] = df_response4.apply(lambda row: row['geometry.coordinates'][0], axis = 1)
df_response4['longitude']

0    -3.663068
1    -3.693897
2    -3.690786
3    -3.692879
4    -3.678921
5    -3.679172
6    -3.693698
7    -3.724914
8    -3.783695
9    -3.700490
10   -3.675164
11   -3.707250
12   -3.630212
13   -3.699310
14   -3.738322
15   -3.703525
16   -3.702410
17   -3.712198
18   -3.700840
Name: longitude, dtype: float64

In [20]:
#free bicipark
df_response4['available_bikes'] = df_response4['total_places']-df_response4['free_places']

In [21]:
#creamos una función que nos cree el enlace a google maps
df_response4['Google Maps'] = df_response4.apply(lambda row: google_maps(row['geometry.coordinates']),axis = 1)

In [22]:
df_response4.head()

,stationId,integratorId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates,latitude,longitude,available_bikes,Google Maps
0,74,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,10,1,0,Point,"[-3.6630679, 40.4846838]",40.484684,-3.663068,0,"https://www.google.com/maps?q=40.4846838,-3.66..."
1,22,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,8,1,1,Point,"[-3.693897, 40.452973]",40.452973,-3.693897,7,"https://www.google.com/maps?q=40.452973,-3.693897"
2,49,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Montalban,"Calle de Montalbán, 5",Madrid,28014,16,10,1,0,Point,"[-3.690786, 40.418146]",40.418146,-3.690786,6,"https://www.google.com/maps?q=40.418146,-3.690786"
3,12,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Almagro,"Calle Almagro, 11",Madrid,28010,16,9,1,0,Point,"[-3.692879, 40.430361]",40.430361,-3.692879,7,"https://www.google.com/maps?q=40.430361,-3.692879"
4,5,9d37a769-4892-4dc4-b838-9f01d0ffd687,Bicipark Recuerdo,"Calle Hiedra, 26",Madrid,28036,13,11,1,0,Point,"[-3.678921, 40.472419]",40.472419,-3.678921,2,"https://www.google.com/maps?q=40.472419,-3.678921"


In [23]:
url = 'https://datos.madrid.es/egob/catalogo/200304-0-centros-culturales.json'
response = requests.get(url)
response.content
json_data = response.json()
json_data.keys()
pull_json = json_data['@graph']
df_pull_json = pd.DataFrame(pull_json)
df_pull_json.head()

,@id,@type,id,title,relation,address,location,organization
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5989871,Auditorio Carmen Laforet (Ciudad Lineal),http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.47967249471556, 'longitude': -...",{'organization-desc': 'Ubicado en el Centro de...
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,2389,Auditorio al aire libre. Parque del Calero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.436300129244366, 'longitude': ...",{'organization-desc': ' Mantenimiento: Dependi...
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,139451,Auditorio y sala de exposiciones Paco de Lucía...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.38493929920335, 'longitude': -...",{'organization-desc': ' Metro de Madrid. Líne...
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,1914,Centro Cultural - Centro Socio Cultural Villa ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.47324881250678, 'longitude': -...",{'organization-desc': ' Metro: Barajas (línea...
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,117288,Centro Cultural Agustín Díaz (Moncloa - Aravaca),http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42754184442827, 'longitude': -...","{'organization-desc': ' Bus: 41, 75', 'accesi..."


In [24]:
#SACAMOS LA LATITUD
def ext_dict_latitud(a):
    return a.get('latitude')
df_pull_json['latitude'] = df_pull_json.apply(lambda row: ext_dict_latitud(row['location']),axis = 1)
df_pull_json['latitude']

0      40.479672
1      40.436300
2      40.384939
3      40.473249
4      40.427542
         ...    
114    40.392458
115    40.393433
116    40.393073
117    40.421218
118    40.382387
Name: latitude, Length: 119, dtype: float64

In [25]:
#SACAMOS LA LONGITUD
def ext_dict_longitude(a):
    return a.get('longitude')
df_pull_json['longitude'] = df_pull_json.apply(lambda row: ext_dict_longitude(row['location']),axis = 1)
df_pull_json['longitude']

0     -3.665724
1     -3.650408
2     -3.764196
3     -3.578491
4     -3.732411
         ...   
114   -3.697207
115   -3.698578
116   -3.699006
117   -3.660765
118   -3.710862
Name: longitude, Length: 119, dtype: float64

In [26]:
df_pull_json

,@id,@type,id,title,relation,address,location,organization,latitude,longitude
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5989871,Auditorio Carmen Laforet (Ciudad Lineal),http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.47967249471556, 'longitude': -...",{'organization-desc': 'Ubicado en el Centro de...,40.479672,-3.665724
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,2389,Auditorio al aire libre. Parque del Calero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.436300129244366, 'longitude': ...",{'organization-desc': ' Mantenimiento: Dependi...,40.436300,-3.650408
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,139451,Auditorio y sala de exposiciones Paco de Lucía...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.38493929920335, 'longitude': -...",{'organization-desc': ' Metro de Madrid. Líne...,40.384939,-3.764196
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,1914,Centro Cultural - Centro Socio Cultural Villa ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.47324881250678, 'longitude': -...",{'organization-desc': ' Metro: Barajas (línea...,40.473249,-3.578491
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,117288,Centro Cultural Agustín Díaz (Moncloa - Aravaca),http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42754184442827, 'longitude': -...","{'organization-desc': ' Bus: 41, 75', 'accesi...",40.427542,-3.732411
...,...,...,...,...,...,...,...,...,...,...
114,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4776440,Medialab en Matadero. Matadero Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.39245798191536, 'longitude': -...",{'organization-desc': 'Medialab Matadero es un...,40.392458,-3.697207
115,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10189992,Nave de Terneras del Centro Cultural Casa del ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.393432756132874, 'longitude': ...",{'organization-desc': ' Metro: Legazpi (línea...,40.393433,-3.698578
116,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11956343,Sala de exposiciones La Lonja. Centro Cultural...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.39307289237465, 'longitude': -...",{'organization-desc': ' Metro: Legazpi (línea...,40.393073,-3.699006
117,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11601456,Salas de ensayo Pau Donés y Luis Eduardo Aute ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42121783502117, 'longitude': -...",{'organization-desc': 'Sala 1: Pau Donés Sala...,40.421218,-3.660765


In [27]:
#CREAMOS NUEVO DATA FRAME PARA LOS LUGARES DE INTERES
df_li = pd.DataFrame() 
df_li['name_li'] = df_pull_json['title']
df_li['latitude_li'] = df_pull_json['latitude']
df_li['longitude_li'] = df_pull_json['longitude']
df_li

,name_li,latitude_li,longitude_li
0,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724
1,Auditorio al aire libre. Parque del Calero,40.436300,-3.650408
2,Auditorio y sala de exposiciones Paco de Lucía...,40.384939,-3.764196
3,Centro Cultural - Centro Socio Cultural Villa ...,40.473249,-3.578491
4,Centro Cultural Agustín Díaz (Moncloa - Aravaca),40.427542,-3.732411
...,...,...,...
114,Medialab en Matadero. Matadero Madrid,40.392458,-3.697207
115,Nave de Terneras del Centro Cultural Casa del ...,40.393433,-3.698578
116,Sala de exposiciones La Lonja. Centro Cultural...,40.393073,-3.699006
117,Salas de ensayo Pau Donés y Luis Eduardo Aute ...,40.421218,-3.660765


In [28]:
#CREAMOS NUEVO DATAFRAME PARA LAS ESTACIONES DE BICIMAD
df_bm = pd.DataFrame() 
df_bm['name_bm'] = df_response3['name']
df_bm['latitude_bm'] = df_response3['latitude']
df_bm['longitude_bm'] = df_response3['longitude']
df_bm['dock_bikes'] = df_response3['dock_bikes']
df_bm['Google Maps'] = df_response3['Google Maps']
df_bm

,name_bm,latitude_bm,longitude_bm,dock_bikes,Google Maps
0,"453 - Avenida del Ensanche de Vallecas, 9",40.370440,-3.612530,22,"https://www.google.com/maps?q=40.37043968,-3.6..."
1,"267 - Paseo de la Chopera, 33",40.395000,-3.700400,19,https://www.google.com/maps?q=40.3950000000000...
2,150 - Castellana frente a Hermanos Pinzón,40.449100,-3.690800,2,"https://www.google.com/maps?q=40.4491,-3.6908"
3,157- Castellana 164,40.459137,-3.689415,7,"https://www.google.com/maps?q=40.4591366,-3.68..."
4,395b - 395 - Guetaria 84b,40.369168,-3.715691,18,"https://www.google.com/maps?q=40.36916772,-3.7..."
...,...,...,...,...,...
583,9 - Plaza de San Miguel,40.415606,-3.709508,8,"https://www.google.com/maps?q=40.4156057,-3.70..."
584,"265 - Calle Puerto Serrano, 1",40.385412,-3.688461,21,"https://www.google.com/maps?q=40.385412,-3.688461"
585,"498 - Calle Alcala, 546",40.443133,-3.617832,2,https://www.google.com/maps?q=40.4431334999999...
586,202 - Parque Santander,40.441300,-3.708300,5,"https://www.google.com/maps?q=40.4413,-3.7083"


In [29]:
#CREAMOS NUEVO DATAFRAME PARA LAS ESTACIONES DE BICIPARK
df_bp = pd.DataFrame() 
df_bp['name_bp'] = df_response4['stationName']
df_bp['latitude_bp'] = df_response4['latitude']
df_bp['longitude_bp'] = df_response4['longitude']
df_bp['available_bikes'] = df_response4['available_bikes']
df_bp['Google Maps'] = df_response4['Google Maps']
df_bp

KeyError: 'Google Maps'

In [30]:
#bicimad a tiempo real df
df_response3
#bicipark a tiempo real df
df_response4
#lugares de interes df
df_pull_json

,@id,@type,id,title,relation,address,location,organization,latitude,longitude
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,5989871,Auditorio Carmen Laforet (Ciudad Lineal),http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.47967249471556, 'longitude': -...",{'organization-desc': 'Ubicado en el Centro de...,40.479672,-3.665724
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,2389,Auditorio al aire libre. Parque del Calero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.436300129244366, 'longitude': ...",{'organization-desc': ' Mantenimiento: Dependi...,40.436300,-3.650408
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,139451,Auditorio y sala de exposiciones Paco de Lucía...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.38493929920335, 'longitude': -...",{'organization-desc': ' Metro de Madrid. Líne...,40.384939,-3.764196
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,1914,Centro Cultural - Centro Socio Cultural Villa ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.47324881250678, 'longitude': -...",{'organization-desc': ' Metro: Barajas (línea...,40.473249,-3.578491
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,117288,Centro Cultural Agustín Díaz (Moncloa - Aravaca),http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42754184442827, 'longitude': -...","{'organization-desc': ' Bus: 41, 75', 'accesi...",40.427542,-3.732411
...,...,...,...,...,...,...,...,...,...,...
114,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,4776440,Medialab en Matadero. Matadero Madrid,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.39245798191536, 'longitude': -...",{'organization-desc': 'Medialab Matadero es un...,40.392458,-3.697207
115,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,10189992,Nave de Terneras del Centro Cultural Casa del ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.393432756132874, 'longitude': ...",{'organization-desc': ' Metro: Legazpi (línea...,40.393433,-3.698578
116,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11956343,Sala de exposiciones La Lonja. Centro Cultural...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.39307289237465, 'longitude': -...",{'organization-desc': ' Metro: Legazpi (línea...,40.393073,-3.699006
117,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11601456,Salas de ensayo Pau Donés y Luis Eduardo Aute ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.42121783502117, 'longitude': -...",{'organization-desc': 'Sala 1: Pau Donés Sala...,40.421218,-3.660765


In [31]:
#CREO UN KEY PARA QUE NO ME DE ERRORES
df_li['key'] = 0
df_bm['key'] = 0
#CREAMOS EL CROS DE LOS LUGARES DE INTERES FRENTE A LAS BICIMAD
df_libm = pd.merge(df_li , df_bm, how = 'outer')
df_libm

,name_li,latitude_li,longitude_li,key,name_bm,latitude_bm,longitude_bm,dock_bikes,Google Maps
0,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,"453 - Avenida del Ensanche de Vallecas, 9",40.370440,-3.612530,22,"https://www.google.com/maps?q=40.37043968,-3.6..."
1,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,"267 - Paseo de la Chopera, 33",40.395000,-3.700400,19,https://www.google.com/maps?q=40.3950000000000...
2,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,150 - Castellana frente a Hermanos Pinzón,40.449100,-3.690800,2,"https://www.google.com/maps?q=40.4491,-3.6908"
3,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,157- Castellana 164,40.459137,-3.689415,7,"https://www.google.com/maps?q=40.4591366,-3.68..."
4,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,395b - 395 - Guetaria 84b,40.369168,-3.715691,18,"https://www.google.com/maps?q=40.36916772,-3.7..."
...,...,...,...,...,...,...,...,...,...
69967,Salón y sala de exposiciones en Junta Municipa...,40.382387,-3.710862,0,9 - Plaza de San Miguel,40.415606,-3.709508,8,"https://www.google.com/maps?q=40.4156057,-3.70..."
69968,Salón y sala de exposiciones en Junta Municipa...,40.382387,-3.710862,0,"265 - Calle Puerto Serrano, 1",40.385412,-3.688461,21,"https://www.google.com/maps?q=40.385412,-3.688461"
69969,Salón y sala de exposiciones en Junta Municipa...,40.382387,-3.710862,0,"498 - Calle Alcala, 546",40.443133,-3.617832,2,https://www.google.com/maps?q=40.4431334999999...
69970,Salón y sala de exposiciones en Junta Municipa...,40.382387,-3.710862,0,202 - Parque Santander,40.441300,-3.708300,5,"https://www.google.com/maps?q=40.4413,-3.7083"


In [32]:
#CREO UN KEY PARA QUE NO ME DE ERRORES
df_li['key'] = 0
df_bp['key'] = 0
#CREAMOS EL CROS DE LOS LUGARES DE INTERES FRENTE A LAS BICIMAD
df_libp = pd.merge(df_li , df_bp, how = 'outer')
df_libp

,name_li,latitude_li,longitude_li,key,name_bp,latitude_bp,longitude_bp,available_bikes
0,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,Bicipark Fuente de la Mora,40.484684,-3.663068,0
1,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,Bicipark Orense,40.452973,-3.693897,7
2,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,Bicipark Montalban,40.418146,-3.690786,6
3,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,Bicipark Almagro,40.430361,-3.692879,7
4,Auditorio Carmen Laforet (Ciudad Lineal),40.479672,-3.665724,0,Bicipark Recuerdo,40.472419,-3.678921,2
...,...,...,...,...,...,...,...,...
2256,Salón y sala de exposiciones en Junta Municipa...,40.382387,-3.710862,0,Bicipark Avenida Portugal,40.411138,-3.738322,0
2257,Salón y sala de exposiciones en Junta Municipa...,40.382387,-3.710862,0,Bicipark Benavente,40.414464,-3.703525,0
2258,Salón y sala de exposiciones en Junta Municipa...,40.382387,-3.710862,0,Bicipark Fuencarral,40.431005,-3.702410,0
2259,Salón y sala de exposiciones en Junta Municipa...,40.382387,-3.710862,0,Bicipark Plaza España,40.422856,-3.712198,2


In [33]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [ ]:
%%time
#CALCULAMOS LAS DISTANCIAS ENTRE LOS LUGARES DE INTERES FRENTE A LOS BICIMAD
df_libm['bmdistance(m)'] = df_libm.apply(lambda row: distance_meters(row['latitude_li'],row['longitude_li'],row['latitude_bm'],row['longitude_bm']),axis=1)
df_libm

In [ ]:
df_libm
df_libm.drop(['key'], axis = 1)

In [ ]:
%%time
#CALCULAMOS LAS DISTANCIAS ENTRE LOS LUGARES DE INTERES FRENTE A LOS BICIPARK
df_libp['bpdistance(m)'] = df_libp.apply(lambda row: distance_meters(row['latitude_li'],row['longitude_li'],row['latitude_bp'],row['longitude_bp']),axis=1)
df_libp

In [ ]:
df_libp
df_libp.drop(['key'], axis = 1)

In [ ]:
#Lo hacemos para todo el data frame con un groupby
distancias_minimas = df_libm.groupby('name_li')['bmdistance(m)'].idxmin()
distancias_minimas

In [ ]:
#seleccionamos las filas correspondientes a estas distancias minimas
final_filas = df_libm.loc[distancias_minimas]
final_filas
final_filas.drop(['key'], axis = 1)

In [ ]:
#Lo hacemos para todo el data frame con un groupby
distancias_minimas_bp = df_libp.groupby('name_li')['bpdistance(m)'].idxmin()
distancias_minimas_bp

In [ ]:
#seleccionamos las filas correspondientes a estas distancias minimas
final_filas_bp = df_libp.loc[distancias_minimas_bp]
final_filas_bp
final_filas.drop(['key'], axis = 1)

In [ ]:
final_filas.to_csv('data/final_filas_libm_live.csv',index = False)

In [ ]:
final_filas_bp.to_csv('data/final_filas_libp_live.csv',index = False)